In [1]:
%load_ext autoreload
%autoreload 2


from src.utils.tsp_loader import load_tsp_matrix

D = load_tsp_matrix("Dane_TSP_76.xlsx")

In [21]:
from src.utils.neighborhoods_numba_delta import neighbor_cost_delta_numba, neighbor_cost_delta_numba_debug
from src.utils.distance import route_length_fast
# ----------------------------------------------------------
# DEBUG — TEST POPRAWNOŚCI DELTA-COST SĄSIEDZTWA
# ----------------------------------------------------------

def debug_test_delta(distance_matrix, n_tests=10000):
    n = distance_matrix.shape[0]
    import numpy as np

    for fn_id in [0, 1, 2]:  # 0=swap, 2=insert (two_opt jest OK)
        print("\nTestuję neighbor_fn_id =", fn_id)

        for _ in range(n_tests):
            # losowa trasa
            route = np.random.permutation(n)

            # pełny koszt
            full_cost = route_length_fast(distance_matrix, route)

            # sąsiad + delta
            new_route, new_cost_delta = neighbor_cost_delta_numba(
                distance_matrix, route, full_cost, fn_id
            )

            full_new_cost = route_length_fast(distance_matrix, new_route)

            if abs(full_new_cost - new_cost_delta) > 1e-6:
                print("❌ BŁĄD w INSERT")
                print("Old route:", route)
                print("New route:", new_route)
                print("delta:", new_cost_delta)
                print("full :", full_new_cost)
                print("diff :", full_new_cost - new_cost_delta)
                return

            # new_route, new_cost_delta, used_i, used_j = neighbor_cost_delta_numba_debug(
            #     distance_matrix, route, full_cost, fn_id
            # )

            # full_new_cost = route_length_fast(distance_matrix, new_route)

            # if abs(full_new_cost - new_cost_delta) > 1e-6:
            #     print("❌ BŁĄD w INSERT")
            #     print("i =", used_i, "j =", used_j)
            #     print("Old route:", route)
            #     print("New route:", new_route)
            #     print("delta:", new_cost_delta)
            #     print("full :", full_new_cost)
            #     print("diff :", full_new_cost - new_cost_delta)
            #     return


        print("✔ OK — brak błędów delta dla fn_id =", fn_id)

debug_test_delta(D)



Testuję neighbor_fn_id = 0
✔ OK — brak błędów delta dla fn_id = 0

Testuję neighbor_fn_id = 1
✔ OK — brak błędów delta dla fn_id = 1

Testuję neighbor_fn_id = 2
✔ OK — brak błędów delta dla fn_id = 2


In [24]:
import numpy as np
import time
from numba import njit

# Benchmark porównujący wydajność delta vs full cost calculation
# dla operatorów sąsiedztwa w TSP


@njit(cache=True)
def neighbor_full_cost(distance_matrix, route, fn_id):
    """
    Generuje sąsiada i oblicza jego koszt od nowa.
    """
    n = len(route)
    
    i = np.random.randint(0, n)
    j = np.random.randint(0, n)
    while i == j:
        j = np.random.randint(0, n)
    
    new_route = route.copy()
    
    if fn_id == 0:  # SWAP
        tmp = new_route[i]
        new_route[i] = new_route[j]
        new_route[j] = tmp
    elif fn_id == 1:  # TWO-OPT
        if i > j:
            i, j = j, i
        new_route[i:j] = new_route[i:j][::-1]
    else:  # INSERT
        a = new_route[i]
        if i < j:
            for k in range(i, j):
                new_route[k] = new_route[k + 1]
            new_route[j] = a
        else:
            for k in range(i, j, -1):
                new_route[k] = new_route[k - 1]
            new_route[j] = a
    
    # obliczamy pełny koszt
    total = 0.0
    for k in range(n - 1):
        total += distance_matrix[new_route[k], new_route[k + 1]]
    total += distance_matrix[new_route[-1], new_route[0]]
    
    return new_route, total


def benchmark_comparison(distance_matrix, n_iterations=10000, warmup=100):
    """
    Porównuje wydajność dwóch podejść:
    - full cost: przeliczanie całej trasy (O(n))
    - delta cost: obliczanie tylko zmiany (O(1))
    """
    n = distance_matrix.shape[0]
    
    print("\n" + "="*60)
    print("Benchmark: Delta vs Full Cost")
    print("="*60)
    print(f"Rozmiar problemu: {n} miast")
    print(f"Iteracji: {n_iterations}")
    print(f"Warmup: {warmup}\n")
    
    from src.utils.neighborhoods_numba_delta import neighbor_cost_delta_numba
    
    results = {}
    operators = {0: "SWAP", 1: "TWO-OPT", 2: "INSERT"}
    
    for fn_id in [0, 1, 2]:
        op_name = operators[fn_id]
        print(f"\n{'-'*60}")
        print(f"Operator: {op_name}")
        print(f"{'-'*60}")
        
        # warmup
        print("Warmup...", end=" ", flush=True)
        route = np.random.permutation(n)
        current_cost = 0.0
        for k in range(n - 1):
            current_cost += distance_matrix[route[k], route[k + 1]]
        current_cost += distance_matrix[route[-1], route[0]]
        
        for _ in range(warmup):
            _, _ = neighbor_full_cost(distance_matrix, route, fn_id)
            _, _ = neighbor_cost_delta_numba(distance_matrix, route, current_cost, fn_id)
        print("done")
        
        # test full cost
        print("\nTest 1: Full cost (całkowite przeliczenie)")
        route = np.random.permutation(n)
        
        start = time.perf_counter()
        for _ in range(n_iterations):
            new_route, new_cost = neighbor_full_cost(distance_matrix, route, fn_id)
            if np.random.random() < 0.5:
                route = new_route
        time_full = time.perf_counter() - start
        
        print(f"  Czas: {time_full:.4f} s")
        print(f"  Per iteracja: {(time_full / n_iterations) * 1e6:.2f} us")
        
        # test delta cost
        print("\nTest 2: Delta cost (tylko zmiana)")
        route = np.random.permutation(n)
        current_cost = 0.0
        for k in range(n - 1):
            current_cost += distance_matrix[route[k], route[k + 1]]
        current_cost += distance_matrix[route[-1], route[0]]
        
        start = time.perf_counter()
        for _ in range(n_iterations):
            new_route, new_cost = neighbor_cost_delta_numba(
                distance_matrix, route, current_cost, fn_id
            )
            if np.random.random() < 0.5:
                route = new_route
                current_cost = new_cost
        time_delta = time.perf_counter() - start
        
        print(f"  Czas: {time_delta:.4f} s")
        print(f"  Per iteracja: {(time_delta / n_iterations) * 1e6:.2f} us")
        
        # wyniki
        speedup = time_full / time_delta
        improvement = ((time_full - time_delta) / time_full) * 100
        
        print(f"\nWynik: delta jest {speedup:.2f}x szybsza ({improvement:.1f}% oszczednosci)")
        
        results[op_name] = {
            'time_full': time_full,
            'time_delta': time_delta,
            'speedup': speedup,
            'improvement': improvement
        }
    
    # podsumowanie
    print(f"\n{'='*60}")
    print("Podsumowanie")
    print(f"{'='*60}\n")
    
    print(f"{'Operator':<12} {'Full [s]':<10} {'Delta [s]':<10} {'Speedup':<10} {'Oszczednosc':<12}")
    print("-"*60)
    
    for op_name in ["SWAP", "TWO-OPT", "INSERT"]:
        r = results[op_name]
        print(f"{op_name:<12} {r['time_full']:<10.4f} {r['time_delta']:<10.4f} "
              f"{r['speedup']:<10.2f}x {r['improvement']:<12.1f}%")
    
    avg_speedup = np.mean([r['speedup'] for r in results.values()])
    print(f"\nSrednie przyspieszenie: {avg_speedup:.2f}x")
    print("="*60 + "\n")
    
    return results


def quick_benchmark(distance_matrix, n_iterations=5000):
    """Szybsza wersja benchmarku z mniejsza liczba iteracji."""
    print("\nSzybki benchmark (mniej iteracji)")
    return benchmark_comparison(distance_matrix, n_iterations=n_iterations, warmup=50)

In [ ]:
results = benchmark_comparison(D, n_iterations=20_000_000)


Benchmark: Delta vs Full Cost
Rozmiar problemu: 77 miast
Iteracji: 20000000
Warmup: 100


------------------------------------------------------------
Operator: SWAP
------------------------------------------------------------
Warmup... done

Test 1: Full cost (całkowite przeliczenie)
  Czas: 30.3747 s
  Per iteracja: 1.52 us

Test 2: Delta cost (tylko zmiana)
  Czas: 27.2344 s
  Per iteracja: 1.36 us

Wynik: delta jest 1.12x szybsza (10.3% oszczednosci)

------------------------------------------------------------
Operator: TWO-OPT
------------------------------------------------------------
Warmup... done

Test 1: Full cost (całkowite przeliczenie)
  Czas: 32.2352 s
  Per iteracja: 1.61 us

Test 2: Delta cost (tylko zmiana)
